In [1]:
import pandas as pd

In [2]:
fileName = 'mpi'
df = pd.read_csv('csvs/' + fileName + '.csv').drop(columns=[
#                                                                 'cpuCnt',
#                                                                 'threadCnt',
                                                                'callCnt'
                                                                ])

In [3]:
def parseFileNameFromPath(val):
    val = str(val)
    return val.split('/')[-1].split('.')[0]
def roundTime(val):
    return round(val, 4)

In [4]:
df.fileName = df.fileName.apply(parseFileNameFromPath)

In [5]:
# only for mpi
df = df[df.cpuCnt != 2]
df = df[df.threadCnt != 1]

In [6]:
df

,fileName,realTime,threadCnt,cpuCnt,algoName
40,saj12,70.899,10,3,mpi
41,saj12,70.672,10,3,mpi
42,saj12,74.108,10,3,mpi
43,saj12,72.350,10,3,mpi
44,saj12,72.259,10,3,mpi
...,...,...,...,...,...
319,saj7,131.826,5,4,mpi
320,saj7,137.657,5,4,mpi
321,saj7,137.541,5,4,mpi
322,saj7,130.209,5,4,mpi


In [7]:
df = df.groupby(['fileName',
                         'cpuCnt',
                         'threadCnt'
           ]).mean()
df.realTime = df.realTime.apply(roundTime)

In [8]:
df

realTime
fileName cpuCnt threadCnt          
saj12    3      5          122.7260
                10          71.7923
                15          63.2767
                20          55.1936
         4      5          125.5993
                10          57.3432
                15          49.2835
                20          44.7386
saj6     3      5           50.8584
                10          33.3417
                15          27.7284
                20          26.3768
         4      5           52.4282
                10          32.2240
                15          28.1418
                20          26.1661
saj7     3      5          132.5276
                10          84.0055
                15          68.5804
                20          64.9596
         4      5          133.1156
                10          84.4819
                15          69.8820
                20          63.2993

In [9]:
df = df.rename(columns={
    'algoName': 'Algoritmus',
    'fileName': 'Název instance',
    'realTime': 'Čas',
    'threadCnt': 'Počet vláken',
    'cpuCnt': 'Počet jader',
    'callCnt': 'Počet volání'
})
df

Čas
fileName cpuCnt threadCnt          
saj12    3      5          122.7260
                10          71.7923
                15          63.2767
                20          55.1936
         4      5          125.5993
                10          57.3432
                15          49.2835
                20          44.7386
saj6     3      5           50.8584
                10          33.3417
                15          27.7284
                20          26.3768
         4      5           52.4282
                10          32.2240
                15          28.1418
                20          26.1661
saj7     3      5          132.5276
                10          84.0055
                15          68.5804
                20          64.9596
         4      5          133.1156
                10          84.4819
                15          69.8820
                20          63.2993

In [10]:
df.to_csv('tablesForReport/' + fileName + '.csv')

### Prepare seq file

In [11]:
df_seq = pd.read_csv('csvs/seq.csv').drop(columns=['cpuCnt', 'threadCnt', 'callCnt'])
df_seq.fileName = df_seq.fileName.apply(parseFileNameFromPath)
df_seq = df_seq.groupby(['fileName']).mean()
df_seq

,realTime
fileName,
saj12,236.836
saj6,59.887
saj7,146.307


In [12]:
seqTime_saj12 = round(df_seq.iloc[0].realTime, 4)
seqTime_saj6 = round(df_seq.iloc[1].realTime, 4)
seqTime_saj7 = round(df_seq.iloc[2].realTime, 4)

In [13]:
def countSn(seqTime, parTime):
    return round(seqTime/parTime, 4)
def countEn(seqTime, parTime, p):
    return round(seqTime/(p * parTime), 4)

In [14]:
Snp = []
Enp = []
for index, row in df.iterrows():
    if index[0] == 'saj12':
        seqTime = seqTime_saj12
    elif index[0] == 'saj6':
        seqTime = seqTime_saj6
    else:
        seqTime = seqTime_saj7

    parTime = round(row['Čas'], 4)
    
    Snp.append(countSn(seqTime, parTime))
    Enp.append(countEn(seqTime, parTime, index[1]))
df['S(n,p)'] = Snp
df['E(n,p)'] = Enp
df = df.drop(columns=['Čas'])
df

S(n,p)  E(n,p)
fileName cpuCnt threadCnt                
saj12    3      5          1.9298  0.6433
                10         3.2989  1.0996
                15         3.7429  1.2476
                20         4.2910  1.4303
         4      5          1.8856  0.4714
                10         4.1301  1.0325
                15         4.8056  1.2014
                20         5.2938  1.3234
saj6     3      5          1.1775  0.3925
                10         1.7962  0.5987
                15         2.1598  0.7199
                20         2.2704  0.7568
         4      5          1.1423  0.2856
                10         1.8585  0.4646
                15         2.1280  0.5320
                20         2.2887  0.5722
saj7     3      5          1.1040  0.3680
                10         1.7416  0.5805
                15         2.1334  0.7111
                20         2.2523  0.7508
         4      5          1.0991  0.2748
                10         1.7318  0.4330
                15         2.0936  0.5234
                20         2.3114  0.5778

In [15]:
df.to_csv('tablesForReport/' + fileName + '-efectivity.csv')